# ルービンの因果モデル

**ルービンの因果モデル**（rubin causal model）は潜在的結果と割当のメカニズムに着目して因果推論を検討するアプローチ。


### 潜在的結果

ある個体が「介入を受けた場合」と「介入を受けなかった場合」の結果を、実際に介入があったかどうかに関わらず定めたものを**潜在的結果**（potential outcome）という。

- **対照群**（control group）：介入を受けなかった（$D=0$の）場合
- **処置群**（treatment group）：介入を受けた（$D=1$の）場合
- 個体$i$が介入を受けなかった時の潜在的結果：$Y_i(0)$
- 個体$i$が介入を受けた時の潜在的結果：$Y_i(1)$


実際の観察結果は$Y_i$とする

$$
Y_i = \left\{
\begin{array}{l}
Y_i(0) \ \text{if } D_i = 0 \\
Y_i(1) \ \text{if } D_i=1
\end{array}
\right.
$$

次のように書くこともできる

$$
Y_i = Y_i(D_i) = Y_i(0)(1-D_i) + Y_i(1)D
$$


### 因果効果の定義

因果の定義の仕方はいろいろあるが、ルービン因果モデルにおいては「ある個体が介入$D$を受けた場合に、受けなかった場合の結果$Y$と比べた結果$Y_1$の差$Y_1 - Y_0$」を「$D$の**因果効果**（causal effect）」とする。なお、因果効果と同様の意味で**処置効果**（treatment effect）という言葉も使われる。

個体$i$の因果効果（**個体処置効果** individual treatment effect） は

$$
\tau_i = Y_i(1)-Y_i(0)
$$

となる。

ただし、実際には実行された処置についての結果しか観測することができないため、$\tau_i$は観測することはできない。このことは**因果推論の根本問題**と呼ばれる。


## SUTVA条件

個体$i$の潜在的結果$Y_i(1), Y_i(0)$が次の2つの条件を満たすとき、**SUTVA**（stable unit treatment value assumption）条件を満たすという（Rubin 1980）。

1. 個体$i$の潜在的結果は、他の個体が受ける処置の値に依存しない
2. 個体$i$に対する処置は1通りに定まる


### 条件1について

条件1は相互干渉がない（no interference）という言い方もされる（Cox 1958）。

相互干渉がある場合、個体$i$の潜在的結果は個体$i$の処置$D_i$だけでは定まらず、$Y_i(D_1, \cdots, D_n)$のように書く必要が出てきてかなり複雑になる。
そのため、条件1は潜在的結果をシンプルに扱うための条件となる。

しかし現実には相互干渉が発生しうる。例えば労働者への教育プログラムの効果を推定する際には、他にもそのプログラムを受けて技能を向上させた労働者が多く現れる場合、労働市場における希少価値は低下し、結果として個体$i$の潜在的結果に影響を与えると考えられる。

### 条件2について

条件2は処置の内容を明確にすることを要請している。

例えば「ダイエットをする」という処置では、具体的に何をしたのかが人によって大きく異なる可能性がある。

